Given that matrix $L$ is a lower triangular matrix 

$$L = \begin{pmatrix} 
l_{11} & 0 & 0 & 0 & 0 \\
l_{21} & l_{22} & 0 & 0 & 0 \\
l_{31} & l_{32} & l_{33} &\cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
l_{n1} & l_{n2} & l_{n3} & \cdots & l_{nn}
\end{pmatrix}
$$

let $x$ and $b$ be vectors of length $n$ then for $Lx =b$ solving for $x$ can be done in $O(n^2)$ complexity.


$l_{11}x_1 = b_1$<br>
$l_{21}x_1 + l_{22}x_2 = b_2$<br>
$l_{31}x_1 + l_{32}x_2 + l_{33}x_3 = b_3$<br>
$\vdots\qquad\vdots\qquad\vdots\qquad\vdots$<br>
$l_{i1}x_1 + l_{i2}x_2 + l_{i3}x_3 + \cdots + l_{ii}x_i= b_i$<br>
$\vdots\qquad\vdots\qquad\vdots\qquad\vdots\qquad\vdots$<br>
$l_{n1}x_1 + l_{n2}x_2 + l_{n3}x_3 + \cdots + l_{ni}x_i + \cdots + l_{nn}x_n= b_i$
<br>
<br>
<br>
To derive the formula stat by solving for $x_1$
$$x_1 = \frac{b_1}{l_{11}}$$
<br>
Knowing $x_1$ means we can solve for $x_2$
$$x_2 = \frac{b_2 - l_{21}x_1}{l_{22}}$$
<br>
now $x_3$
$$x_3 = \frac{b_3 - (l_{31}x_1 + l_{32}x_2)}{l_{33}}$$
<br>
Continuing in this fashion to solve for the $x_i$ gives us a simple algorithm to solve for any lower triangular system
$$x_i = \frac{b_i - (l_{i1}x_1 + l_{i2}x_2 + l_{i3}x_3 + \cdots + l_{i(i-1)}x_{(i-1)}}{l_{ii}}$$
<br>
$$ = \Bigg(b_i - \sum^{i-1}_{j=1}l_{ij}x_{j}\Bigg)\Bigg/l_{ii}$$

In [7]:
function row_forwardSub(A, b)
	x = zeros(size(b))
	x[1] = b[1] / A[1,1]
	for i in 2:size(b)[1]
		for j in 1:i-1
			x[i] += A[i,j]*x[j]
		end
		x[i] = (b[i] - x[i])/A[i,i]
	end
	return x
	end;

# The Block Matrix Approach

<b>Since Julia is a column major it might be possible to pick up speed in evaluating lower triangualr matrices column wise. The approch to doing so will be a recursive algorithm

$$\begin{pmatrix} 
l_{11} & 0 & 0 & 0 & 0 \\
l_{21} & l_{22} & 0 & 0 & 0 \\
l_{31} & l_{32} & l_{33} &\cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
l_{n1} & l_{n2} & l_{n3} & \cdots & l_{nn}
\end{pmatrix}
\begin{pmatrix}
x_1 \\
x_2 \\
x_3 \\
\ddots \\
x_n
\end{pmatrix}
=
\begin{pmatrix}
b_1 \\
b_2 \\
b_3 \\
\ddots \\
b_n
\end{pmatrix}
$$

<br>

The first step is to turn $L$ into a block matrix and block the two vectors $x$ and $b$

<br>

$$\begin{pmatrix} 
l_{11} & \hat{0} \\
\hat{l} & \hat{L}
\end{pmatrix}
\begin{pmatrix}
x_1 \\
\hat{x}
\end{pmatrix}
=
\begin{pmatrix}
b_1 \\
\hat{b}
\end{pmatrix}
$$

<br>

where $l_{11}$ is the first entry in the triangular matrix, $\:\: \hat{l}$ is vector of length $(n - 1)$, $\:\:\hat{0}$ is column vector of length $(n - 1)$,$\:\:$and finaly $\hat{L}$ is an $(n - 1) \times (n-1)$ matrix. solving for $x_1$ is still just about free $x_1 = b_1\big/l_{11}$

<br>
<br>

$$\hat{l}x_1 + \hat{L}\hat{x} = \hat{b}$$
<br>
$$\hat{L}\hat{x} = \hat{b} - \hat{l}x_1$$

<br>
<br>

With that $\hat{L}$ can be turned into a block matrix, and $l_{22}$ will be the first entry and $x_2$ can be solved for as easy as $x_2 = b_2\big/l_{22}$

<br>
<br>

$$\hat{l}x_2 + \hat{L}\hat{x} = \hat{b}$$
<br>
$$\hat{L}\hat{x} = \hat{b} - \hat{l}x_2$$

<br>
<br>

I will continue in this fashion until all $x$'s have been solved for

The function

Pre loop:
<br>
Instead of allocating a vector of length $n$ to store values of $x$ a copy if the vector $b$ will be made this is so values of the vector can be manipulated without overwriting the original vector it will also store the each value of $x$ that is solved for.


main loop:
<br>
solve for the $i^{th}$ value of $x$ then store that value in the $i^{th}$ slot of the vector $b$

$$x_{i} = b_i \bigg/ l_{ii}$$

inner loop:
<br>
The nested loop is for updating each remaining value of $b$ which represents $\hat b$ the vector $\hat{l}$ will be made from the $i^{th}$ column of $L$ and restricting the range of the inner loop starting it at $i +1$ 

 $$b_j = b_j - l_{ji}x_i$$



In [4]:
function col_forwardSub(L,b)
	b_copy = copy(b)
	for i in 1:size(b_copy)[1]
		b_copy[i] = b_copy[i]/L[i,i]
		for j in i+1:size(b_copy)[1]
			b_copy[j] -= L[j,i] * b_copy[i]
		end
	end
	return b_copy	
	end;

Finaly to see the preformance of the builtin row, and column
I will use the BenchmarkTools libray

In [1]:
using LinearAlgebra
using BenchmarkTools
n = 1000;
L = LowerTriangular(rand(n,n));
b = rand(n,1);

In [13]:
@benchmark L\b

BenchmarkTools.Trial: 
  memory estimate:  7.94 KiB
  allocs estimate:  1
  --------------
  minimum time:     405.666 μs (0.00% GC)
  median time:      431.286 μs (0.00% GC)
  mean time:        455.590 μs (0.00% GC)
  maximum time:     1.847 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [14]:
@benchmark row_forwardSub(L,b)

BenchmarkTools.Trial: 
  memory estimate:  7.94 KiB
  allocs estimate:  1
  --------------
  minimum time:     794.523 μs (0.00% GC)
  median time:      815.607 μs (0.00% GC)
  mean time:        843.676 μs (0.00% GC)
  maximum time:     3.315 ms (0.00% GC)
  --------------
  samples:          5840
  evals/sample:     1

In [15]:
@benchmark col_forwardSub(L,b)

BenchmarkTools.Trial: 
  memory estimate:  7.94 KiB
  allocs estimate:  1
  --------------
  minimum time:     544.303 μs (0.00% GC)
  median time:      549.759 μs (0.00% GC)
  mean time:        565.346 μs (0.00% GC)
  maximum time:     1.666 ms (0.00% GC)
  --------------
  samples:          8697
  evals/sample:     1